## Attempt 1 to optimise the NN

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"],axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced use the variable name `application_types_to_replace`
application_types_to_replace = ["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

In [4]:
value_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = value_counts.loc[value_counts < 1000].index.tolist()

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
no = ["0"]
low = ["1-9999","10000-24999","25000-99999"]
medium = ["100000-499999","1M-5M"]
high = ["5M-10M","10M-50M","50M+"]

for value in no:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(value,"Nothing")

for value in low:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(value,"Low")

for value in medium:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(value,"Medium")

for value in high:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(value,"High")

In [6]:
cat_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT']
cat_df = pd.get_dummies(application_df, columns=cat_columns)
cat_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_High,INCOME_AMT_Low,INCOME_AMT_Medium,INCOME_AMT_Nothing
0,1,5000,1,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
1,1,108590,1,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [7]:
# Split our preprocessed data into our features and target arrays
y = cat_df["IS_SUCCESSFUL"].values
X = cat_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 7
hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 312       
                                                                 
 dense_1 (Dense)             (None, 7)                 63        
                                                                 
 dense_2 (Dense)             (None, 7)                 56        
                                                                 
 dense_3 (Dense)             (None, 1)                 8         
                                                                 
Total params: 439
Trainable params: 439
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 891us/step - loss: 2.1767 - accuracy: 0.5962
Epoch 2/100
804/804 [==============================] - 1s 921us/step - loss: 1.0190 - accuracy: 0.6863
Epoch 3/100
804/804 [==============================] - 1s 892us/step - loss: 0.6114 - accuracy: 0.7111
Epoch 4/100
804/804 [==============================] - 1s 856us/step - loss: 0.5808 - accuracy: 0.7173
Epoch 5/100
804/804 [==============================] - 1s 874us/step - loss: 0.5994 - accuracy: 0.7130
Epoch 6/100
804/804 [==============================] - 1s 904us/step - loss: 0.5820 - accuracy: 0.7172
Epoch 7/100
804/804 [==============================] - 1s 889us/step - loss: 0.5658 - accuracy: 0.7248
Epoch 8/100
804/804 [==============================] - 1s 871us/step - loss: 0.5637 - accuracy: 0.7259
Epoch 9/100
804/804 [==============================] - 1s 861us/step - loss: 0.5626 - accuracy: 0.7253
Epoch 10/100
804/804 [==============================] - 1s 931us/step - l

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5567 - accuracy: 0.7250 - 284ms/epoch - 1ms/step
Loss: 0.5566607713699341, Accuracy: 0.7250145673751831


## Attempt 2 to Optimise

In [13]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.drop(columns=["EIN","NAME","USE_CASE"],axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,N,5000,1
1,T3,Independent,C2000,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Trust,1,100000-499999,N,142590,1


In [14]:
# Choose a cutoff value and create a list of application types to be replaced use the variable name `application_types_to_replace`
application_types_to_replace = ["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


classifications_to_replace = value_counts.loc[value_counts < 1000].index.tolist()

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [15]:
cat_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT','AFFILIATION']
cat_df = pd.get_dummies(application_df, columns=cat_columns)
cat_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [16]:
# Split our preprocessed data into our features and target arrays
y = cat_df["IS_SUCCESSFUL"].values
X = cat_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 7
hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 8)                 312       
                                                                 
 dense_5 (Dense)             (None, 7)                 63        
                                                                 
 dense_6 (Dense)             (None, 7)                 56        
                                                                 
 dense_7 (Dense)             (None, 1)                 8         
                                                                 
Total params: 439
Trainable params: 439
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 906us/step - loss: 1.2905 - accuracy: 0.6626
Epoch 2/100
804/804 [==============================] - 1s 968us/step - loss: 0.8210 - accuracy: 0.7051
Epoch 3/100
804/804 [==============================] - 1s 925us/step - loss: 0.6518 - accuracy: 0.6521
Epoch 4/100
804/804 [==============================] - 1s 902us/step - loss: 0.5902 - accuracy: 0.7168
Epoch 5/100
804/804 [==============================] - 1s 854us/step - loss: 0.5766 - accuracy: 0.7233
Epoch 6/100
804/804 [==============================] - 1s 865us/step - loss: 0.5731 - accuracy: 0.7245
Epoch 7/100
804/804 [==============================] - 1s 862us/step - loss: 0.5694 - accuracy: 0.7258
Epoch 8/100
804/804 [==============================] - 1s 863us/step - loss: 0.5676 - accuracy: 0.7257
Epoch 9/100
804/804 [==============================] - 1s 848us/step - loss: 0.5639 - accuracy: 0.7272
Epoch 10/100
804/804 [==============================] - 1s 851us/step - l

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5592 - accuracy: 0.7262 - 251ms/epoch - 937us/step
Loss: 0.559160590171814, Accuracy: 0.7261807322502136


## Attempt 3 to optimise NN

In [21]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.drop(columns=["EIN","NAME"],axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [22]:
# Define the bin edges
bins = [0, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000, 50000000, 100000000]

# Bin the ASK_AMT column using the cut function
application_df['ASK_AMT_binned'] = pd.cut(application_df['ASK_AMT'], bins=bins)
application_df.drop(columns=['ASK_AMT'],axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_binned
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1,"(0, 5000]"
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,"(100000, 500000]"
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,"(0, 5000]"
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,"(5000, 10000]"
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,"(100000, 500000]"


In [23]:
# Choose a cutoff value and create a list of application types to be replaced use the variable name `application_types_to_replace`
application_types_to_replace = ["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


classifications_to_replace = value_counts.loc[value_counts < 1000].index.tolist()

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [24]:
cat_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT','AFFILIATION', 'ASK_AMT_binned','USE_CASE']
cat_df = pd.get_dummies(application_df, columns=cat_columns)
cat_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,"ASK_AMT_binned_(500000, 1000000]","ASK_AMT_binned_(1000000, 5000000]","ASK_AMT_binned_(5000000, 10000000]","ASK_AMT_binned_(10000000, 50000000]","ASK_AMT_binned_(50000000, 100000000]",USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [25]:
# Split our preprocessed data into our features and target arrays
y = cat_df["IS_SUCCESSFUL"].values
X = cat_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="PReLU")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="PReLU"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 8)                 432       
                                                                 
 dense_19 (Dense)            (None, 8)                 80        
                                                                 
 dense_20 (Dense)            (None, 7)                 63        
                                                                 
 dense_21 (Dense)            (None, 1)                 8         
                                                                 
Total params: 583
Trainable params: 583
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6123 - accuracy: 0.6860
Epoch 2/100
804/804 [==============================] - 1s 915us/step - loss: 0.5669 - accuracy: 0.7261
Epoch 3/100
804/804 [==============================] - 1s 944us/step - loss: 0.5617 - accuracy: 0.7270
Epoch 4/100
804/804 [==============================] - 1s 912us/step - loss: 0.5592 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 1s 925us/step - loss: 0.5564 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 1s 932us/step - loss: 0.5548 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 1s 981us/step - loss: 0.5523 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 1s 999us/step - loss: 0.5531 - accuracy: 0.7308
Epoch 9/100
804/804 [==============================] - 1s 970us/step - loss: 0.5510 - accuracy: 0.7312
Epoch 10/100
804/804 [==============================] - 1s 936us/step - los

In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')